                                          PROJECT AI 
                                   TWITTER SENTIMENT ANALYSIS
                                    MUHAMMAD RAFEH(FL-21513)
                                          ABDULLAH SHAH
                                           AMMAN IKRAM
                                 SUBMITTED TO : DR. SANIA ARSHAD
                                          BSSE-5B (MOR)

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
! pip install kaggle

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rafeh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#load from csv to pandas dataframe
twitter_data=pd.read_csv('train.csv', encoding='ISO-8859-1')

In [4]:
#check number of rows and column
twitter_data.shape

(1599999, 6)

In [5]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [6]:
#renaming the column and printing data frame again
column_name=['target','id','date','flag','user','text']
twitter_data=pd.read_csv('train.csv',names=column_name, encoding='ISO-8859-1')
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [7]:
#count number of mising values in data set
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [8]:
#check the distribution of target columns
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [9]:
#check number of rows and column
twitter_data.shape

(1600000, 6)

In [10]:
#replacing 4 witgh 1 in target column to refer to positive
twitter_data.replace({'target':{4:1}},inplace=True)

In [11]:
#check the distribution of target columns
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0 --> negative tweets,
1 --> positive tweets

In [12]:
port_stem=PorterStemmer()

In [13]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [14]:
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)

In [15]:
#printing stemmed content
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [16]:
#printing target column
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [17]:
#seperating values and label
x=twitter_data['stemmed_content'].values
y=twitter_data['target'].values

In [18]:
print(x)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [19]:
print(y)

[0 0 0 ... 1 1 1]


SPLITING DATA INTO TRAIN DATA AND TEST DATA

In [20]:
#split data into train test data
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

In [21]:
print(x.shape,x_train.shape,x_test.shape)

(1600000,) (1280000,) (320000,)


In [22]:
print(x_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [23]:
print(x_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [27]:
import pickle
#converting textual data into numerical data and saving the model using pickle
vectorizer=TfidfVectorizer()

x_train=vectorizer.fit_transform(x_train)
x_test=vectorizer.transform(x_test)
filename='trained_vectorizer_model.sav'
pickle.dump(vectorizer,open(filename,'wb'))

In [25]:

print(x_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [26]:
print(x_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [29]:
import pickle
loaded_model = pickle.load(open('C:/Users/rafeh/OneDrive/Desktop/ai project/trained_twitter_model.sav', 'rb'))

x_test_transformed = vectorizer.transform(x_test)

y_pred = loaded_model.predict(x_test_transformed)

TRAINING THE MODEL

Logistic regression

In [34]:
model=LogisticRegression(max_iter=1000)

In [35]:
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

MODEL EVALUATION

In [36]:
#accuracy score on train data
x_train_prediction=model.predict(x_train)
training_data_accuracy= accuracy_score(y_train,x_train_prediction)

In [37]:
print ('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.79871953125


In [38]:
#accuracy score on test data
x_test_prediction=model.predict(x_test)
test_data_accuracy= accuracy_score(y_test,x_test_prediction)

In [39]:
print ('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.79871953125


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Generate classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:")
print(conf_matrix)


Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.76      0.77    160000
           1       0.77      0.80      0.78    160000

    accuracy                           0.78    320000
   macro avg       0.78      0.78      0.78    320000
weighted avg       0.78      0.78      0.78    320000

Confusion Matrix:
[[121246  38754]
 [ 32708 127292]]


SAVING TRAINED MODEL

In [40]:
import pickle

In [41]:
filename='trained_twitter_model.sav'
pickle.dump(model,open(filename,'wb'))

**UPLOAD the Trained MODEL**

In [2]:
import pickle
loaded_model = pickle.load(open('C:/Users/rafeh/OneDrive/Desktop/ai project/trained_twitter_model.sav', 'rb'))


# positive tweeet given for this prediction : "Just finished my project, and it turned out amazing! Feeling so accomplished! 🎉"


In [34]:
stemmer=PorterStemmer()
# test on new data
User_input = input("Enter a tweet: ")
#stemming
stemmed_content=stemming(User_input)
# Uploading vectorizer model 
vectorizer = pickle.load(open('C:/Users/rafeh/OneDrive/Desktop/ai project/trained_vectorizer_model.sav', 'rb'))
x_new=vectorizer.transform([stemmed_content])


prediction=loaded_model.predict(x_new)
print(prediction)
if prediction==0:
    print("Negative")
else:
    print("Positive")


[1]
Positive
